In [2]:
import sys
sys.path.insert(0, 'C:\\Users\\Paarth Tandon\\Desktop\\repos\\personal-music-gen')

In [3]:
from audiocraft.models import MusicGen
from personal_musicgen.data.datasets import AudioDataset
from personal_musicgen.model_utils import *

import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.cuda.amp import GradScaler
from torch.utils.data import DataLoader, Subset
import torchaudio

In [4]:
data = AudioDataset('../data/chunks_no_voice/')
model = MusicGen.get_pretrained('small')
model.lm = model.lm.to(torch.float32)

c:\users\paarth tandon\desktop\repos\personal-music-gen\libs\audiocraft\audiocraft\models\musicgen.py:116: UserWarning: MusicGen pretrained model relying on deprecated checkpoint mapping. Please use full pre-trained id instead: facebook/musicgen-small
  warnings.warn(
c:\Users\Paarth Tandon\Desktop\repos\personal-music-gen\.venv\lib\site-packages\torch\nn\utils\weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [5]:
model.device

device(type='cuda', index=0)

In [6]:
dataloader = DataLoader(Subset(data, list(range(100))), batch_size=1)
acc_steps = 4
optimizer = AdamW(
    model.lm.parameters(),
    lr=1e-5,
    betas=(0.9, 0.95),
    weight_decay=0.1,
)
scaler = torch.cuda.amp.GradScaler()

In [9]:
from tqdm import tqdm


def train_step(
        model: MusicGen,
        optimizer: AdamW,
        scaler: GradScaler,
        dataloader: DataLoader,
        grad_acc_steps: int
) -> dict:
    device = model.device
    
    total_loss = 0

    for i, (audio_fns, label_fns) in tqdm(enumerate(dataloader), total=len(dataloader)):
        codes_l = []
        text_l = []

        for audio_fn, label_fn in zip(audio_fns, label_fns):
            codes = encode_audio(model, audio_fn)
            codes_l.append(codes)
            with open(label_fn, 'r') as label_f:
                text_l.append(label_f.read().strip())
        
        codes = torch.cat(codes_l, dim=0).to(device)

        attributes, _ = model._prepare_tokens_and_attributes(text_l, None)
        conditional_vector = get_contitional_vector(model, attributes)

        with torch.autocast(device_type="cuda", dtype=torch.float16):
            lm_output = model.lm.compute_predictions(
                codes=codes,
                conditions=[],
                condition_tensors=conditional_vector
            )

            loss = compute_masked_loss(lm_output, codes)
            scaler.scale(loss).backward()
            
            total_loss += loss.item()
            print(loss.item())

            if (i + 1) % grad_acc_steps == 0:
                scaler.unscale_(optimizer)
                torch.nn.utils.clip_grad_norm_(model.lm.parameters(), 1.0)
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()

    return {
        'loss': total_loss / len(dataloader)
    }

In [10]:
train_step(model, optimizer, scaler, dataloader, acc_steps)

  1%|          | 1/100 [00:05<09:41,  5.88s/it]

3.240298271179199


  2%|▏         | 2/100 [00:09<07:11,  4.40s/it]

3.163398027420044


  3%|▎         | 3/100 [00:12<06:08,  3.80s/it]

...

3.9527554512023926


 36%|███▌      | 36/100 [02:18<03:41,  3.46s/it]

3.7286298274993896


 36%|███▌      | 36/100 [02:21<04:10,  3.92s/it]


KeyboardInterrupt: 